### Task 3 of the course Introduction to Machine Learning

Import necessary modules

In [45]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

Unzip the folder containing all the images

In [46]:
local_runtime = False
partial_training = False
model_version = 'V2B6'

if local_runtime :
  foldername='Tasks/Task3/task3_data'
  path=foldername+'/'
else :
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  filename='drive/MyDrive/food.zip'
  foldername='data'
  path=foldername+'/'
  !unzip -q -n $filename -d $foldername
  !cp -n drive/MyDrive/test_triplets.txt drive/MyDrive/train_triplets.txt -d $path

Mounted at /content/drive


Detect if the GPU is available to later move the computations on it.

In [47]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


**Randomize the labels** 

Currently, in the train set : 
*   Image A : it's the reference
*   Image B : the closest image to A
*   Image C : the one different from A

So the label is 1 (i.e. Image B) for all the train triplets -> the best prediction is to always predict 1

We shuffle B and C images in the triplet (while keeping track of it) so that the labels are uniformely distributed between 0 and 1



In [48]:
df_train = pd.read_csv(path+'train_triplets.txt', header=None, delim_whitespace=True)
print(df_train.head(10))
"""
np.random.seed(0) # to always generate the same "random" shuffling
train_label = np.random.randint(2, size=(df_train.shape[0]))
pd.DataFrame(train_label).to_csv(path+'train_labels.csv',header=None, index=None)
print(train_label[0:10])

# Invert image B and C if label is 0
mask=~train_label.astype(bool) 
df_train.loc[mask,1],df_train.loc[mask,2] = df_train.iloc[mask][2],df_train.iloc[mask][1]
print(df_train.head(10))
"""
if partial_training:
  df_train=df_train.iloc[:64*30]
  train_label=train_label[:64*30]


      0     1     2
0  2461  3450  2678
1  2299  2499  4987
2  4663  1056  3029
3  4532  1186  1297
4  3454  3809  2204
5  2851  3047   524
6  1168   262  2663
7  2134  4969  1153
8  2741  2851  1518
9  4668  2527  3481


In [49]:
class TripletDataset(Dataset):

    def __init__(self, df_idx, img_ds, train=False, labels=None):
        """
        Args:
            df_labels (pd.DataFrame): Panda DataFrame containing the triplets label
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df_idx = df_idx
        self.img_ds = img_ds
        self.n_ftrs = self.img_ds.shape[1]
        self.train  = train
        self.labels = labels

    def __len__(self):
        return len(self.df_idx)

    def __getitem__(self, row):
        
        img_tri=torch.empty((0,self.n_ftrs), dtype=torch.float)
        for i in self.df_idx.iloc[row]:
          img_tri = torch.cat((img_tri, self.img_ds[i].unsqueeze(dim=0)),0)
        if self.train :
          return img_tri, self.labels[row]
        else :
          return img_tri

In [50]:
"""
train_label=torch.from_numpy(train_label).long()

# convert labels to one-hot encoding
train_label = torch.nn.functional.one_hot(train_label, num_classes=2).float()

print(train_label[:10])
"""

'\ntrain_label=torch.from_numpy(train_label).long()\n\n# convert labels to one-hot encoding\ntrain_label = torch.nn.functional.one_hot(train_label, num_classes=2).float()\n\nprint(train_label[:10])\n'

In [51]:
len(df_train)

59515

In [52]:
# split between training and validation sets
X_train, X_test, y_train, y_test = train_test_split(df_train, train_label, test_size=20.0/len(df_train), random_state=24)


#########################################
#                                       #
#   !!! extremely small test size !!!   #
#                                       #
#########################################


In [53]:
img_ftrs=torch.load('drive/MyDrive/img_efficientnet.pt')
nb_ftrs=img_ftrs.shape[1]
ds_train=TripletDataset(X_train,img_ftrs, True, y_train)
ds_test=TripletDataset(X_test,img_ftrs, True, y_test)

**Copied from tutorial !!!**

In [54]:
# setup train and test loader
train_loader = DataLoader(ds_train, batch_size=64, shuffle=True,num_workers=4)
test_loader = DataLoader(ds_test, batch_size=128, shuffle=True,num_workers=2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Create new ConvNet with using EfficientNet or ResNet as the basis and then add a few layers.

In [198]:
# Version 1 of the network

class CNN(torch.nn.Module):

    def __init__(self, num_ftrs):
        super().__init__()

        self.num_ftrs = num_ftrs
        self.leaky_relu = torch.nn.LeakyReLU()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.layer = nn.Sequential(self.dropout,
                                   nn.Linear(self.num_ftrs, self.num_ftrs),
                                   self.dropout,
                                   self.leaky_relu,
                                   nn.Linear(self.num_ftrs, 1000),
                                   )
        self.conv1 = torch.nn.Conv1d(in_channels=1, out_channels=16, 
                                     kernel_size=2, stride=1)
        self.conv2 = torch.nn.Conv1d(in_channels=16, out_channels=32, 
                                     kernel_size=2, stride=1)
        self.pool = torch.nn.MaxPool2d(1, 2)
        self.fc0 = nn.Sequential(self.dropout,
                                 nn.Linear(499*2*32, 240),
                                 self.dropout,
                                 self.leaky_relu,
                                 nn.Linear(240, 32))
        
        #self.fc0 = nn.Sequential(self.conv1,self.leaky_relu,
        #                         self.conv2,self.leaky_relu)

        

        self.fc1 = nn.Linear(32,128)
        self.fc2 = nn.Linear(128,96)
        self.fc3 = nn.Linear(96,20)
        self.sim = nn.CosineSimilarity(dim=1, eps=1e-08)
        self.rse = nn.PairwiseDistance(p=2)



# IDEA: identity last layer of pretrained model and to fc2


    def forward(self, x):
        
        x = self.layer(x)
        
        x = x.unsqueeze(dim=1)

        x = self.leaky_relu(self.conv1(x))
        
        x = self.leaky_relu(self.conv2(x))
        
        x = self.dropout(torch.flatten(x,start_dim = 1))

        x = self.fc0(x)

        x = self.leaky_relu(self.dropout(self.fc1(x)))

        x = self.leaky_relu(self.dropout(self.fc2(x)))

        logits = self.fc3(x)

        return logits

In [199]:
# move the model to the GPU
model = CNN(nb_ftrs).to(device)

In [200]:
#loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)

In [201]:
# code from pytorch tutorial !!!

def accuracy(logits: torch.Tensor, label: torch.Tensor) -> torch.Tensor:
  # computes the classification accuracy
  correct_label = torch.argmax(logits, axis=-1) == torch.argmax(label, axis=-1)
  assert correct_label.shape == (logits.shape[0],)
  acc = torch.mean(correct_label.float())
  assert 0. <= acc <= 1.
  return acc

def evaluate(model: torch.nn.Module) -> torch.Tensor:
  # goes through the test dataset and computes the test accuracy
  model.eval()  # bring the model into eval mode
  with torch.no_grad():
    acc_cum = 0.0
    num_eval_samples = 0
    for x_batch_test, y_label_test in test_loader:
      x_batch_test, y_label_test = x_batch_test.to(device), y_label_test.to(device)
      batch_size = x_batch_test.shape[0]
      num_eval_samples += batch_size
      acc_cum += accuracy(model(x_batch_test), y_label_test) * batch_size
    avg_acc = acc_cum / num_eval_samples
    assert 0 <= avg_acc <= 1
    return avg_acc

In [202]:
# Setup the optimizer (adaptive learning rate method)
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

epoch_init=0
if epoch_init>0 :
  checkpoint = torch.load(f'model_{model_version}_epoch_{epoch_init}.pt')
  model.load_state_dict(checkpoint['model_state_dict'])

N_EPOCH=100

for epoch in range(epoch_init+1,epoch_init+1+N_EPOCH):
  # reset statistics trackers
  train_loss_cum = 0.0
  acc_cum = 0.0
  num_samples_epoch = 0
  t = time.time()

  # Go once through the training dataset (-> epoch)
  for x_batch, y_batch in tqdm(train_loader):
      # zero grads and put model into train mode
      optim.zero_grad()
      model.train()
    

      # move data to GPU
      x_batch, y_batch = x_batch.to(device), y_batch.to(device)

      # forward pass
      #logits = model(x_batch)
      #loss = loss_fn(logits, y_batch)
      
      #forward pass triplet loss
      a = model(x_batch[:,0,:])
      p = model(x_batch[:,1,:])
      n = model(x_batch[:,2,:])
      loss = loss_fn(a, p, n)

      # backward pass and gradient step
      loss.backward()
      optim.step()
      """
      # keep track of train stats
      num_samples_batch = x_batch.shape[0]
      num_samples_epoch += num_samples_batch
      train_loss_cum += loss * num_samples_batch
      acc_cum += accuracy(logits, y_batch) * num_samples_batch
      """
  """    
  # average the accumulated statistics
  avg_train_loss = train_loss_cum / num_samples_epoch
  avg_acc = acc_cum / num_samples_epoch
  test_acc = evaluate(model)
  """
  epoch_duration = time.time() - t
  
  # print some infos
  """
  print(f'Epoch {epoch} | Train loss: {train_loss_cum.item():.4f} | '
  f' Train accuracy: {avg_acc.item():.4f} | Test accuracy: {test_acc.item():.4f} |'
  f' Duration {epoch_duration:.2f} sec')
  """
  print(f' Duration {epoch_duration:.2f} sec')
  # save checkpoint of model
  if epoch % 5 == 0 and epoch > 0:
      save_path = f'model_{model_version}_epoch_{epoch}.pt'
      torch.save({'epoch': epoch,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optim.state_dict()},
                 save_path)
      print(f'Saved model checkpoint to {save_path}')

  0%|          | 0/930 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 930/930 [00:34<00:00, 27.25it/s]


 Duration 34.13 sec


100%|██████████| 930/930 [00:34<00:00, 27.21it/s]


 Duration 34.19 sec


100%|██████████| 930/930 [00:34<00:00, 26.90it/s]


...

 Duration 35.08 sec


100%|██████████| 930/930 [00:34<00:00, 27.16it/s]


 Duration 34.26 sec


100%|██████████| 930/930 [00:34<00:00, 27.15it/s]


 Duration 34.27 sec
Saved model checkpoint to model_V2B6_epoch_100.pt


In [203]:
epoch_load=75
if epoch_load>0 :
  # load model from checkpoint
  model_loaded = CNN(nb_ftrs)
  checkpoint = torch.load(f'model_{model_version}_epoch_{epoch_load}.pt')
  model_loaded.load_state_dict(checkpoint['model_state_dict'])
  model = model_loaded.to(device)


In [211]:

df_pred = pd.read_csv(path+'test_triplets.txt', header=None, delim_whitespace=True)
print(df_pred.head(10))


ds_pred=TripletDataset(df_pred,img_ftrs, False)

# setup train and test loader
pred_loader = DataLoader(ds_pred, batch_size=1, shuffle=False, num_workers=2)

pred_label=[]

model.eval()  # bring the model into eval mode
with torch.no_grad():
  for x_batch in tqdm(pred_loader):
    x_batch = x_batch.to(device)
    a_test = model(x_batch[:,0,:]).cpu()
    p_test = model(x_batch[:,1,:]).cpu()
    n_test = model(x_batch[:,2,:]).cpu()
    label = int(torch.cdist(a_test, p_test) < torch.cdist(a_test, n_test))
    #pred_label.append(np.argmax(model(x_batch).cpu(),axis=1).tolist())
    pred_label.append(label)

      0     1     2
0  9896  9640  9177
1  6592  9283  7104
2  8655  6174  6400
3  9223  8187  8678
4  7317  5392  9470
5  9175  9545  8854
6  6660  5883  6814
7  6506  7220  5847
8  8388  8012  6407
9  9990  7083  9257


100%|██████████| 59544/59544 [05:31<00:00, 179.62it/s]


In [209]:
print(pred_label)

[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 

In [208]:
flat_list = [item for sublist in pred_label for item in sublist]
flat_list[:10]

TypeError: ignored

In [213]:
#pred_label_flat = np.asarray(flat_list)
pd.DataFrame(pred_label).to_csv(f'drive/MyDrive/label_prediction_{model_version}.csv', index=None, header=None)